## Introduction
#### Large portion of Data Analysis is taken up by preparation: loading, cleaning, transforming and rearranging.
#### These tasks take up more than 80% of an analyst's time. This is because the way the data is stored in files or databases is not in the right format.
#### Researchers prefer to do ad-hoc processing of data from one form to another using languages like R, Python, etc.
#### pandas with built-in Python features provides high-level, flexible abd fast set of tools that enables you to manipulate data into right form.

## Handling missing data
#### The way missing data is represented in pandas is imperfect but functional for lot of users.
#### For numeric data, pandas uses floating-point value NaN. It is called a Sentinel value and can be easily detected.

In [3]:
import pandas as pd
import numpy as np

string_data = pd.Series(['aardvak', 'artichoke', np.nan, 'avocado'])
string_data

0      aardvak
1    artichoke
2          NaN
3      avocado
dtype: object

In [4]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

#### This convention of NaN was adopted from R, where missing values are refered as NA.
#### In statistics, NA may either be data that does not exist or data that was not observed.
#### During cleaning data, we should also analyse the missing data to identify data collection problems or potential bias due to missing data.

In [5]:
string_data[0] = None

string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### Filtering 'Out' Missing Data
#### We always have the option to filter out missing data by hand using 'isnull' and boolean indexing.
#### The 'dropna' function can be pretty useful too. For a Series it returns the Series with only non-null data and index values.
#### For DataFrame, it is a bit complex. dropna by default will drop any row that contains even 1 missing value. By passing "how='all'" will target rows with all NAs.
#### To drop columns, pass 'axis=1'.

In [6]:
from numpy import nan as NA

data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [7]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

In [8]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                   [NA, NA, NA], [NA, 6.5, 3.]])

cleaned = data.dropna()
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [9]:
cleaned

,0,1,2
0,1.0,6.5,3.0


In [10]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [11]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [12]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


#### Another DataFrame cleaning method concerns with time series data.
#### To keep only rows with certain number of observations, use the 'thresh' argument.
#### Any row that contains number of NAs greater than or equal thresh will be eliminated.

In [13]:
df = pd.DataFrame(np.random.randn(7,3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA

df

,0,1,2
0,-1.152841,NaN,NaN
1,1.392019,NaN,NaN
2,-0.138522,NaN,-0.801143
3,1.727443,NaN,0.011845
4,-0.008346,-0.910039,1.191962
5,0.846771,1.635657,0.192299
6,0.611303,-0.422075,0.599414


In [14]:
df.dropna()

,0,1,2
4,-0.008346,-0.910039,1.191962
5,0.846771,1.635657,0.192299
6,0.611303,-0.422075,0.599414


In [15]:
df.dropna(thresh=2)

,0,1,2
2,-0.138522,NaN,-0.801143
3,1.727443,NaN,0.011845
4,-0.008346,-0.910039,1.191962
5,0.846771,1.635657,0.192299
6,0.611303,-0.422075,0.599414


### Filling In Missing Data
#### Rather than removing NAs and discarding important information in the same rows, we can also fill in the NAs in different ways.
#### The 'fillna' is a workhorse function, where the constant we pass replaces missing values.
#### If we call fillna with a dict, we can fill different value for each column.

In [16]:
df.fillna(0)

,0,1,2
0,-1.152841,0.000000,0.000000
1,1.392019,0.000000,0.000000
2,-0.138522,0.000000,-0.801143
3,1.727443,0.000000,0.011845
4,-0.008346,-0.910039,1.191962
5,0.846771,1.635657,0.192299
6,0.611303,-0.422075,0.599414


In [17]:
df.fillna({1:0.5, 2: 0})

,0,1,2
0,-1.152841,0.500000,0.000000
1,1.392019,0.500000,0.000000
2,-0.138522,0.500000,-0.801143
3,1.727443,0.500000,0.011845
4,-0.008346,-0.910039,1.191962
5,0.846771,1.635657,0.192299
6,0.611303,-0.422075,0.599414


#### By default it returns a new object, but we can modify it to change in-place.
#### The interpolation methods used for reindexing like 'ffill' can also be used with fillna.
#### It allows you to do lots of creative things, like filling with mean or median values.

In [18]:
_ = df.fillna(0, inplace=True)

df

,0,1,2
0,-1.152841,0.000000,0.000000
1,1.392019,0.000000,0.000000
2,-0.138522,0.000000,-0.801143
3,1.727443,0.000000,0.011845
4,-0.008346,-0.910039,1.191962
5,0.846771,1.635657,0.192299
6,0.611303,-0.422075,0.599414


In [19]:
df = pd.DataFrame(np.random.randn(6,3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,-1.196822,-1.728530,0.011651
1,0.635502,-1.557243,1.468730
2,-0.357786,NaN,0.239709
3,-0.456500,NaN,-0.379100
4,-0.511740,NaN,NaN
5,-0.005963,NaN,NaN


In [20]:
df.fillna(method='ffill')

,0,1,2
0,-1.196822,-1.728530,0.011651
1,0.635502,-1.557243,1.468730
2,-0.357786,-1.557243,0.239709
3,-0.456500,-1.557243,-0.379100
4,-0.511740,-1.557243,-0.379100
5,-0.005963,-1.557243,-0.379100


In [21]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,-1.196822,-1.728530,0.011651
1,0.635502,-1.557243,1.468730
2,-0.357786,-1.557243,0.239709
3,-0.456500,-1.557243,-0.379100
4,-0.511740,NaN,-0.379100
5,-0.005963,NaN,-0.379100


In [22]:
data = pd.Series([1., NA, 3.5, NA, 7])

data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

## Data Transformation
#### Till now we have seen methods for rearranging data.
#### Transformation involves filtering, cleaning and other different functions.

### Removing Duplicates
#### The DataFrame method 'duplicated' returns boolean Series indicating if each row is a duplicate (i.e. observed in a previous row) or not.
#### Similarly, 'drop_duplicates' returns DataFrame where 'duplicated' array is False.

In [23]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                    'k2': [1,1,2,3,3,4,4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [24]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [25]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


#### Both the above methods by default consider all of the columns. You can also specify any subset of the DataFrame to detect duplicates.
#### By default, both keep the first observation in case of duplicates. We can specify "keep='last'" to instead keep the last observation.

In [26]:
data['v1'] = range(7)
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


In [27]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### Transforming Data Using a Function or Mapping
#### We sometimes need to make transformations based on the values present in an array, Series or column in a DataFrame.
#### We can use the map method with a function or dict-like object having the mapping to add or change a column.
#### Sometimes the column that we base our mapping on may have varying case from our map. In such a case, we can convert all the values to lowercase. Or just pass a function that does it for us.

In [28]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                             'Pastrami', 'corned beef', 'Bacon',
                            'pastrami', 'honey ham', 'nova lox'],
                    'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [29]:
meat_to_animal = {
    'bacon': 'pig',
    'pulled pork': 'pig',
    'pastrami': 'cow',
    'corned beef': 'cow',
    'honey ham': 'pig',
    'nova lox': 'salmon'
}

In [30]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [31]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [32]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### Replacing Values
#### The fillna method is a special case of more general values replacement.
#### The map function modifies a subset of values but, 'replace' provides simpler and more flexible way to do so.
#### Passing the sentinel (or garbage) value followed by the replcae value will create a new object with the values replaced.
#### If we want in-place replacement, use "inplace=True".

In [33]:
data = pd.Series([1., -999., 2., -999., 3.])
data

0      1.0
1   -999.0
2      2.0
3   -999.0
4      3.0
dtype: float64

In [34]:
data.replace(-999, np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    3.0
dtype: float64

#### To replace multiple values with a single value, pass a list followed by substitute value.
#### To have different replacements for different values, pass list of substitutes.
#### You can also pass a dict as argument to replace multiple substitutes.
#### NOTE - 'data.replace' is different from 'data.str.replace'. The latter is for element-wise string substitution.

In [35]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    3.0
dtype: float64

In [36]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    3.0
dtype: float64

In [37]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    3.0
dtype: float64

### Renaming Axis Indexes
#### Just like values, axis labels can also be transformed by a function or mapping to produce differntly labeled objects.
#### We can also modify axes in-place without any new data structure.

In [38]:
data = pd.DataFrame(np.arange(12).reshape((3,4)),
                   index = ['Ohio', 'Colorado', 'New York'],
                   columns = ['one', 'two', 'three', 'four'])

In [39]:
transform = lambda x: x[:4].upper()

data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [40]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


#### To get a transformed version of a dataset without modifying the original, use 'rename'.
#### It can also be used in conjunction with a dict-like object providing new values for subset of the axis labels.
#### It saves you from copying DataFrame manuallyand then assigning it index and columns. To modify in-place, use parameter 'inplace=True'.

In [41]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [42]:
data.rename(index={'OHIO':"INDIANA"},
           columns = {'three':'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [43]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### Discretization and Binning
#### Continuous data is always discretized or seperated into 'bins' for analysis.
#### To bin a set of continuous data, use the 'cut' method from pandas.
#### In below example, we are binnning set of gaes into groups 18 to 25, 26 to 35, 36 to 60 and 61 nd older.

In [44]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]

cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

#### pandas returns a special Categorical object from cuts function.
#### The output describes the bins that each of the element is in. You can treat it like a bin name for each element.
#### Internally, the output contains a categories array specifying distinct category names along with a labeling for the 'ages' data in the 'codes' attribute.

In [45]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [46]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]]
              closed='right',
              dtype='interval[int64]')

In [47]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

#### The interval system for cut is consistent with the mathematical notation. A parenthesis means that the side is open and a square bracket means that it is closed (inclusive).
#### We can changes which side is closed by passing 'right=False'.
#### We can have our own bin names by passing a list or array to the labels option.

In [48]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [49]:
group_names = ['Youth', 'YoungAdult', 'MiddleAges', 'Senior']
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAges, MiddleAges, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAges < Senior]

#### Instead of specifying specific intervals, we can just pass an integer to get equal length bins of the same number based on the max and min value of the data.
#### The 'precision' parameter limits decimal precision in the values. 'precision=2' limits decimal precision to 2 digits.

In [50]:
data = np.random.rand(20)

pd.cut(data, 4, precision=2)

[(0.51, 0.74], (0.28, 0.51], (0.51, 0.74], (0.055, 0.28], (0.51, 0.74], ..., (0.28, 0.51], (0.055, 0.28], (0.055, 0.28], (0.28, 0.51], (0.055, 0.28]]
Length: 20
Categories (4, interval[float64]): [(0.055, 0.28] < (0.28, 0.51] < (0.51, 0.74] < (0.74, 0.97]]

#### cut has a closely related function - 'qcut' that bins data based on sample quantiles.
#### Based on distribution, using cut will not usually result in each bin have the same number of data points.
#### But as qcut uses sample quantiles, you wil rougjly obtain equal-size bins.
#### We can even pass our own quantiles to qcut.

In [51]:
data = np.random.randn(1000)
cats = pd.qcut(data, 4)
cats

[(-3.413, -0.659], (0.0656, 0.778], (0.0656, 0.778], (0.778, 3.118], (-0.659, 0.0656], ..., (0.0656, 0.778], (-0.659, 0.0656], (-0.659, 0.0656], (0.0656, 0.778], (0.778, 3.118]]
Length: 1000
Categories (4, interval[float64]): [(-3.413, -0.659] < (-0.659, 0.0656] < (0.0656, 0.778] < (0.778, 3.118]]

In [52]:
pd.value_counts(cats)

(0.778, 3.118]      250
(0.0656, 0.778]     250
(-0.659, 0.0656]    250
(-3.413, -0.659]    250
dtype: int64

In [53]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(-3.413, -1.172], (0.0656, 1.378], (0.0656, 1.378], (0.0656, 1.378], (-1.172, 0.0656], ..., (0.0656, 1.378], (-1.172, 0.0656], (-1.172, 0.0656], (0.0656, 1.378], (1.378, 3.118]]
Length: 1000
Categories (4, interval[float64]): [(-3.413, -1.172] < (-1.172, 0.0656] < (0.0656, 1.378] < (1.378, 3.118]]

### Detecting and Filtering Outliers
#### Filtering and Transforming outliers is mostly a matter of applying array operations.
#### To find values exceeding a threshold, just use boolean indexing with other functions like 'abs()' based on requirement.
#### To get all rows having at least one value exceed a threshold, use the 'any(1)' method.
#### Values can also be set based on these criteria. So you can cap values based on an interval or threshold.
#### You can also use the 'np.sign()' function to get 1 and -1 where the data is positive or negative respectively.

In [54]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.014491,0.014646,-0.012298,0.039840
std,1.010034,1.010854,1.019774,0.967757
min,-3.701866,-3.387856,-3.039110,-3.390256
25%,-0.657859,-0.654879,-0.685004,-0.619386
50%,0.066684,-0.001560,0.020953,0.058701
75%,0.692534,0.713840,0.628787,0.696540
max,3.058465,3.226376,4.065409,3.088712


In [55]:
col = data[2]
col[np.abs(col) > 3]

360    3.099117
559    4.065409
583   -3.039110
976    3.180584
Name: 2, dtype: float64

In [56]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
186,3.058465,0.016649,-1.722464,1.367502
228,-2.056428,-1.475965,-0.003494,-3.390256
360,0.124732,-0.404802,3.099117,-0.237581
388,-0.253419,3.105240,0.078645,-0.148384
470,-3.701866,-2.025745,0.727206,-1.468939
472,-0.484421,-3.387856,-0.112233,-0.068768
508,0.323240,3.226376,-1.808320,-0.094250
559,-0.579666,1.194872,4.065409,-1.209842
583,0.658055,1.219842,-3.039110,0.021802
636,-3.087210,1.114326,-0.585877,0.080978


In [57]:
# Capping outside -3 to 3
data[np.abs(data) > 3] = np.sign(data) * 3 
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.015455,0.014702,-0.013604,0.040396
std,1.006524,1.008608,1.015092,0.965337
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.657859,-0.654879,-0.685004,-0.619386
50%,0.066684,-0.001560,0.020953,0.058701
75%,0.692534,0.713840,0.628787,0.696540
max,3.000000,3.000000,3.000000,3.000000


In [58]:
np.sign(data).head()

,0,1,2,3
0,1.0,1.0,-1.0,-1.0
1,-1.0,-1.0,1.0,-1.0
2,-1.0,1.0,1.0,-1.0
3,1.0,1.0,1.0,1.0
4,1.0,1.0,-1.0,1.0


### Permutation and Random Sampling
#### We can easily perform Permutation (randomly reordering) on a Series or rows of a DataFrame using 'numpy.random.permutation'.
#### Calling it with length of the axis you want to permute cerates an array of integers indicating the new ordering.
#### we can then use the same array in an iloc-based indexing or with an equivalent 'take' function.

In [59]:
df = pd.DataFrame(np.arange(5*4).reshape((5,4)))
sampler = np.random.permutation(5)
sampler

array([1, 0, 4, 2, 3])

In [60]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [61]:
df.take(sampler)

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
4,16,17,18,19
2,8,9,10,11
3,12,13,14,15


#### Use the 'sample' method to select random subset without replacement.
#### To generate subset with replacement (i.e. repeat choices) pass 'replace=True'.

In [62]:
df.sample(n=3)

,0,1,2,3
1,4,5,6,7
2,8,9,10,11
4,16,17,18,19


In [63]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

0    5
1    7
0    5
4    4
4    4
4    4
4    4
3    6
0    5
2   -1
dtype: int64

### Computing Indicator / Dummy Variables
#### Another transformation is to convert a categorical variable into a 'dummy' or 'indicator' matrix.
#### If a column has k distinct values, we can derive a matrix or a DataFrame with k columns all containing 1s and 0s.
#### pandas has the 'get_dummies' function to do this.
#### You may want to add a prefix to the columns in the indicator DataFrame. This DataFrame can be merged with other data. get_dummies has the 'prefix' argument to do this.

In [64]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                  'data1': range(6)})
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [65]:
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


#### But if a row belongs to multiple categories, then dummying it becomes complicated.
#### Adding indicator variable to a column like genre involves a bit of data wrangling.
#### First we get all the unique categorical values in the DataFrame.

In [66]:
mnames = ['movie_id', 'title', 'genres']

movies = pd.read_table('datasets/movielens/movies.dat', sep='::',
                      header=None, names=mnames)
movies[:10]

C:\Users\adity\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [67]:
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))
    
genres = pd.unique(all_genres)
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

#### One way to get this would be to start with a DataFrame of all zeros.
#### The iterate through each row and set the entry in each row of 'dummies' to 1. For this we use 'dummies.columns' to compute column indices for each category.
#### The we use iloc to set values based on those indices. After that we combine this indicator DataFrame with original DataFrame.

In [68]:
zero_matrix = np.zeros((len(movies), len(genres)))
dummies = pd.DataFrame(zero_matrix, columns=genres)

In [69]:
gen = movies.genres[0]
gen.split('|')

['Animation', "Children's", 'Comedy']

In [70]:
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2], dtype=int64)

In [71]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [72]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Adventure                                0
Genre_Fantasy                                  0
Genre_Romance                                  0
Genre_Drama                                    0
Genre_Action                                   0
Genre_Crime                                    0
Genre_Thriller                                 0
Genre_Horror                                   0
Genre_Sci-Fi                                   0
Genre_Documentary                              0
Genre_War                                      0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Film-Noir                                0
Genre_Western       

#### NOTE - For larger data, the above method for creating indicator method is not speedy. It would be better to write a lower-level function that writes directly to NumPy array and then wraps the result in a DataFrame.
#### Another useful method in statistical applications is to combine the dummy function with discretization function like cut. This shows if a value is present in a bin or not.
#### For below example we will use random seed to make the example more deterministic.

In [73]:
np.random.seed(12345)
values = np.random.rand(10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [74]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


## String Manipulation
#### Python is a popular raw data manipulation language due to its ease of use for string and text processing.
#### Simple text operations can be done using String object's built-in methods.
#### For more complex pattern matching and text manipulations, we can use regular expressions.
#### pandas enable us to apply both string and regex functions on whole arrays of data.

### String Object Methods
#### In most string manipulation scenarios, inbuilt string methods are mostly sufficient.
#### A string can be broken based on a seperator using 'split'.
#### It is often combined with 'strip' to trim out whitespace, including line breaks.
#### Substrings can be concatenated together using the '+' operator.
#### A more faster and pythonic way to do so is to pass a list or tuple of substrings to the 'join' method on the 'stitching' string.

In [78]:
val = 'a,b,   guido'

val.split(',')

['a', 'b', '   guido']

In [79]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [80]:
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

In [81]:
'::'.join(pieces)

'a::b::guido'

#### The 'in' keyword is the best way to detect substring. Although, 'index' and 'find' can also be used.
#### There is 1 major difference between 'find' and 'index'. 'index' returns an Exception if substring is not found. 'find' returns a -1.
#### 'count' returns number of occurences of a particular substring.
#### 'replace' substitutes occurence of one pattern for another. It is commonly used to delete patterns by passing an empty string as replacement.

In [83]:
'guido' in val

True

In [84]:
val.index(',')

1

In [85]:
val.find(':')

-1

In [86]:
# val.index(':')

ValueError: substring not found

In [87]:
val.count(',')

2

In [88]:
val.replace(',', '::')

'a::b::   guido'

In [89]:
val.replace(',', '')

'ab   guido'

### Regular Expression
#### It is a flexible way to search or match (often complex) string patterns in text.
#### The expression 'regex' is a string formed by 'regular expression'.
#### Python has built-in 're' module for applying regular expressions to strings.
#### This module works in 3 categories, all are related : 'pattern matching', 'substitution' and 'splitting'.
#### We first describe a pattern that we want to locate in text. This can be used for many purposes.    

#### Once we define a regex, we can use it to split a text using 'split'.
#### In this command, the regex is first compiled and then its split method is called on to the passed text.
#### We can compile regex on our own using 're.compile' which will form a reusable regex object.
#### Compiling regex beforehand is highly recommended if we have to apply it to many strings. This will save CPU cycles.
#### NOTE - To avoid using escape sequences in regex, use raw string literals. eg - r'C:\x' instead of 'c:\\x'.

In [92]:
import re

text = "foo bar\t baz   \tqux"
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

In [94]:
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

#### We can use the 'findall' function to list all the matching patterns to the regex.
#### To get only the first match in a string, we can use 'search' method.
#### 'match' only matches at the beginning of the string i.e. it checks if the string startes with the regex.
#### Using 'IGNORECASE'  makes the regex case-insensitive.

In [96]:
regex.findall(text)

[' ', '\t ', '   \t']

In [100]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""

pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'
regex = re.compile(pattern, flags=re.IGNORECASE)

#### 'findall' returns a list of matching pattern.
#### 'search' returns a special match object for the first matching pattern. It can only tell us the start and end positions of the pattern in a string.
#### 'match' will return None if the pattern does not exist in the string.
#### 'sub' will return a new string with occurences of the pattern replaced by a new pattern.

In [102]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [103]:
m = regex.search(text)
m

<_sre.SRE_Match object; span=(5, 20), match='dave@google.com'>

In [104]:
text[m.start():m.end()]

'dave@google.com'

In [105]:
print(regex.match(text))

None


In [106]:
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



#### To find the patterns and at the same time segment it into different parts, put parentheses around the parts of the pattern to segment.
#### In this case, 'findall' will return a list of tuples, when the pattern has groups.
#### 'match' will return a tuple of the pattern components with its 'groups' method.
#### 'sub' can also be used to access the groups in each match with special symbols '\1', '\2'. Here '\1' corresponds to first matched group.

In [108]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [109]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [111]:
m = regex.match('wes@bright.net')
m.groups()

('wes', 'bright', 'net')

In [112]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



### Vectorized String Functions in pandas
#### The presence of missing data in columns makes data cleaning in datasets very difficult.
#### We could apply string or regex methods using lambda or other functions to each value using data.map, but this will fail on the NA values.
#### To avoid this Series has array-oriented methods for string operations that skip NA values.
#### They can be accessed using the 'str' attribute.

In [116]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [117]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

#### We can check if a string contains a pattern using the 'str.contains' function.
#### We can use regex too, with options like 'IGNORECASE'.

In [119]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [120]:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [121]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

#### To retrieve elements from Vector, we can either use 'str.get' or 'index'.
#### To access elements in embedded list, we pass an index to either of these functions.
#### We can also slice strings with this syntax.

In [123]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [124]:
matches.str.get(1)

Dave    NaN
Steve   NaN
Rob     NaN
Wes     NaN
dtype: float64

In [125]:
matches.str[0]

Dave    NaN
Steve   NaN
Rob     NaN
Wes     NaN
dtype: float64

In [126]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object